In [1]:
from ase.visualize import view
%matplotlib inline

In [19]:
Atomsl = ase.io.read('first_trials/coronene.990000.cfg')
Atomse = ase.io.read('first_trials/coronene.10000.cfg')
Atomsm = ase.io.read('first_trials/coronene.500000.cfg')

In [3]:
arr = Atoms.get_positions()
arr = np.broadcast_to(arr, (arr.shape[0], arr.shape[0], arr.shape[1]))
arr = np.sum(arr**2, axis=2)**0.5
arr.shape

(576, 576)

In [106]:
def get_bond_graph(atoms, bond_length=1.5):
    coords = atoms.get_positions()
    coords = np.broadcast_to(coords, (coords.shape[0], coords.shape[0], coords.shape[1]))
    distances = coords - np.transpose(coords, (1, 0, 2))
    del coords
    box_params = np.broadcast_to(atoms.get_cell().lengths(), distances.shape)
    distances = np.minimum(np.abs(distances), np.abs(box_params - np.abs(distances)))
    del box_params
    distances = np.sum(distances**2, axis=len(distances.shape)-1)**0.5
    adj_matrix = distances
    del distances
    adj_matrix[adj_matrix > bond_length] = 0
    bond_graph = nx.from_numpy_matrix(adj_matrix)
    
    return bond_graph    

In [107]:
view(Atomse, viewer='ngl')

In [108]:
bond_graph = get_bond_graph(Atoms)

In [109]:
nx.algorithms.components.number_connected_components(bond_graph)

106

In [110]:
np.count_nonzero(Atoms.get_atomic_numbers()==6)

384

In [111]:
def get_atom_idxs(atoms, atom_type):
    return np.nonzero(atoms.get_atomic_numbers()==atom_type)[0]

In [112]:
get_atom_idxs(Atoms, atom_type=6)

array([  1,   2,   4,   6,   7,   8,   9,  11,  12,  13,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  28,  30,  31,  33,  35,  36,
        37,  39,  41,  42,  44,  46,  47,  51,  52,  53,  57,  58,  59,
        60,  61,  62,  63,  64,  65,  67,  69,  74,  75,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  89,  90,  93,  94,  96,  98,
        99, 100, 101, 102, 103, 104, 106, 107, 108, 110, 111, 116, 117,
       118, 119, 122, 123, 124, 125, 126, 128, 129, 130, 133, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 152, 153, 154, 155, 156, 157,
       158, 160, 161, 162, 164, 166, 167, 168, 169, 172, 173, 178, 179,
       180, 184, 185, 187, 188, 197, 198, 199, 201, 202, 204, 207, 210,
       211, 214, 215, 216, 217, 220, 223, 224, 225, 228, 229, 230, 231,
       232, 233, 234, 235, 236, 238, 239, 240, 241, 242, 243, 245, 252,
       253, 254, 255, 256, 257, 258, 259, 260, 263, 264, 265, 266, 267,
       268, 269, 273, 275, 276, 279, 282, 283, 284, 287, 288, 28

In [113]:
bond_graph = get_bond_graph(Atoms)
len(list(bond_graph.neighbors(2)))
get_atom_idxs(Atoms, atom_type=6)

array([  1,   2,   4,   6,   7,   8,   9,  11,  12,  13,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  28,  30,  31,  33,  35,  36,
        37,  39,  41,  42,  44,  46,  47,  51,  52,  53,  57,  58,  59,
        60,  61,  62,  63,  64,  65,  67,  69,  74,  75,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  89,  90,  93,  94,  96,  98,
        99, 100, 101, 102, 103, 104, 106, 107, 108, 110, 111, 116, 117,
       118, 119, 122, 123, 124, 125, 126, 128, 129, 130, 133, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 152, 153, 154, 155, 156, 157,
       158, 160, 161, 162, 164, 166, 167, 168, 169, 172, 173, 178, 179,
       180, 184, 185, 187, 188, 197, 198, 199, 201, 202, 204, 207, 210,
       211, 214, 215, 216, 217, 220, 223, 224, 225, 228, 229, 230, 231,
       232, 233, 234, 235, 236, 238, 239, 240, 241, 242, 243, 245, 252,
       253, 254, 255, 256, 257, 258, 259, 260, 263, 264, 265, 266, 267,
       268, 269, 273, 275, 276, 279, 282, 283, 284, 287, 288, 28

In [114]:
def calc_carb_hybr(atoms):
    bond_graph = get_bond_graph(atoms)
    carbon_idxs = get_atom_idxs(atoms, atom_type=6)
    print(type(carbon_idxs))
    sp1 = []
    sp2 = []
    sp3 = []
    others = []
    for carbon in carbon_idxs:
        if len(list(bond_graph.neighbors(carbon))) == 2: sp1.append(carbon)
        elif len(list(bond_graph.neighbors(carbon))) == 3: sp2.append(carbon)
        elif len(list(bond_graph.neighbors(carbon))) == 4: sp3.append(carbon)
        else: others.append(carbon)
    
    sp1 = np.array(sp1)
    sp2 = np.array(sp2)
    sp3 = np.array(sp3)
    others = np.array(others)
    
    return (sp1, sp2, sp3, others)

In [115]:
calc_carb_hybr(Atoms)

<class 'numpy.ndarray'>


(array([  1,   2,   4,   6,   7,  11,  12,  13,  18,  19,  22,  25,  28,
         30,  42,  46,  47,  53,  57,  58,  60,  64,  65,  74,  80,  81,
         85,  86,  90,  93,  96,  99, 102, 104, 106, 107, 110, 117, 119,
        122, 128, 129, 133, 135, 137, 138, 139, 142, 143, 158, 161, 167,
        168, 179, 187, 198, 201, 202, 207, 211, 214, 217, 224, 225, 231,
        235, 239, 240, 245, 252, 255, 256, 258, 259, 260, 263, 264, 267,
        282, 289, 290, 291, 292, 293, 312, 314, 315, 316, 317, 318, 320,
        326, 327, 333, 336, 338, 339, 341, 343, 344, 346, 347, 348, 349,
        351, 353, 354, 356, 359, 361, 364, 365, 368, 373, 381, 382, 385,
        387, 392, 394, 395, 400, 405, 410, 414, 416, 417, 419, 422, 423,
        427, 432, 436, 439, 441, 445, 452, 459, 460, 461, 462, 463, 465,
        470, 475, 476, 478, 482, 483, 484, 486, 487, 490, 492, 494, 497,
        500, 505, 507, 509, 511, 512, 515, 517, 518, 519, 529, 530, 532,
        537, 538, 539, 540, 542, 544, 548, 552, 553

In [48]:
hyb = []
for i in range(len(Atoms.get_atomic_numbers())):
    if Atoms.get_atomic_numbers()[i] == 6:
        count = 0
        for j in range(len(Atoms.get_atomic_numbers())):
               x0 = Atoms.get_positions()[i][0]
               y0 = Atoms.get_positions()[i][1]
               z0 = Atoms.get_positions()[i][2]
               x = Atoms.get_positions()[j][0]
               y = Atoms.get_positions()[j][1]
               z = Atoms.get_positions()[j][2]
               if in_cube(x0,y0,z0,x,y,z,1.8) == True:
                   r = np.sqrt((x-x0)**2 + (y-y0)**2 + (z-z0)**2)
                   if r<=1.8:
                       count+=1
            
        hyb.append([i,count-1])
    

NameError: name 'in_cube' is not defined

In [17]:
def return_hybridisation(Atoms):
    Dist = Atoms.get_all_distances(mic=True)
    hybr = []
    for i in range(len(Atoms.get_atomic_numbers())):
        if Atoms.get_atomic_numbers()[i] == 6:
            count = 0
            for j in range(len(Dist[i])):
                if Dist[i][j] < 1.8:
                    count += 1
            hybr.append([i,count-1])
    for a in range(len(hybr)):
        if hybr[a][1] == 0:
            hybr[a][1] = 'no_hybr'
        elif hybr[a][1] == 1:
            hybr[a][1] = 'no_hybr'
        elif hybr[a][1] == 2:
            hybr[a][1] = 'sp'
        elif hybr[a][1] == 3:
            hybr[a][1] = 'sp2'
        elif hybr[a][1] ==  4:
            hybr[a][1] = 'sp3'
        else: 
            hybr[a][1] = '???'
    return hybr
    

In [20]:
early = return_hybridisation(Atomse)
mid = return_hybridisation(Atoms)
late = return_hybridisation(Atomsl)

In [21]:
early

[[1, 'sp2'],
 [3, 'sp2'],
 [5, 'sp2'],
 [6, 'sp2'],
 [7, 'sp2'],
 [8, 'sp2'],
 [9, 'sp2'],
 [11, 'sp2'],
 [12, 'sp2'],
 [14, 'sp2'],
 [15, 'sp2'],
 [16, 'sp2'],
 [17, 'sp2'],
 [18, 'sp2'],
 [19, 'sp2'],
 [20, 'sp2'],
 [22, 'sp2'],
 [25, 'sp2'],
 [27, 'sp2'],
 [28, 'sp2'],
 [29, 'sp2'],
 [30, 'sp2'],
 [31, 'sp2'],
 [32, 'sp2'],
 [34, 'sp2'],
 [36, 'sp2'],
 [37, 'sp2'],
 [38, 'sp2'],
 [40, 'sp2'],
 [44, 'sp2'],
 [45, 'sp2'],
 [46, 'sp2'],
 [48, 'sp2'],
 [49, 'sp2'],
 [51, 'sp2'],
 [52, 'sp2'],
 [53, 'sp2'],
 [56, 'sp2'],
 [57, 'sp2'],
 [58, 'sp2'],
 [59, 'sp2'],
 [60, 'sp2'],
 [61, 'sp2'],
 [62, 'sp2'],
 [63, 'sp2'],
 [64, 'sp2'],
 [65, 'sp2'],
 [67, 'sp2'],
 [69, 'sp2'],
 [70, 'sp2'],
 [71, 'sp2'],
 [73, 'sp2'],
 [74, 'sp2'],
 [76, 'sp2'],
 [78, 'sp2'],
 [79, 'sp2'],
 [85, 'sp2'],
 [86, 'sp2'],
 [87, 'sp2'],
 [88, 'sp2'],
 [89, 'sp2'],
 [93, 'sp2'],
 [94, 'sp2'],
 [97, 'sp2'],
 [98, 'sp2'],
 [99, 'sp2'],
 [100, 'sp2'],
 [101, 'sp2'],
 [102, 'sp2'],
 [103, 'sp2'],
 [104, 'sp2'],
 [105, '

In [22]:
late

[[1, 'sp2'],
 [2, 'sp2'],
 [4, 'sp2'],
 [6, 'sp'],
 [7, 'sp2'],
 [8, 'sp2'],
 [9, 'sp2'],
 [11, 'sp2'],
 [12, 'sp2'],
 [13, 'sp2'],
 [16, 'sp2'],
 [17, 'sp2'],
 [18, 'sp2'],
 [19, 'sp2'],
 [20, 'sp2'],
 [21, 'sp2'],
 [22, 'sp2'],
 [23, 'sp2'],
 [24, 'sp2'],
 [25, 'sp2'],
 [28, 'sp2'],
 [30, 'sp2'],
 [31, 'sp2'],
 [33, 'sp2'],
 [35, 'sp'],
 [36, 'sp2'],
 [37, 'sp2'],
 [39, 'sp2'],
 [41, 'sp2'],
 [42, 'sp'],
 [44, 'sp'],
 [46, 'sp2'],
 [47, 'sp2'],
 [51, 'sp'],
 [52, 'sp'],
 [53, 'sp'],
 [57, 'sp'],
 [58, 'sp2'],
 [59, 'sp2'],
 [60, 'sp'],
 [61, 'sp2'],
 [62, 'sp2'],
 [63, 'sp3'],
 [64, 'sp2'],
 [65, 'sp2'],
 [67, 'sp2'],
 [69, 'sp2'],
 [74, 'sp2'],
 [75, 'sp2'],
 [78, 'sp2'],
 [79, 'sp2'],
 [80, 'sp3'],
 [81, 'sp2'],
 [82, 'sp2'],
 [83, 'sp2'],
 [84, 'sp2'],
 [85, 'sp2'],
 [86, 'sp2'],
 [87, 'sp2'],
 [89, 'sp2'],
 [90, 'sp2'],
 [93, 'sp'],
 [94, 'sp2'],
 [96, 'sp'],
 [98, 'sp2'],
 [99, 'sp'],
 [100, 'sp'],
 [101, 'sp2'],
 [102, 'sp'],
 [103, 'sp2'],
 [104, 'sp2'],
 [106, 'sp2'],
 [107, 